In [2]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import uuid
import difflib
import html5lib

In [3]:
player_index = pd.read_csv('../updated_datasets/player_index.csv')
player_index = player_index[player_index.columns[1:]]
player_index

,Player,UUID
0,Kevin Garnett,4035d838-5ed7-4507-86ad-0488a9b9c358
1,Shaquille O'Neal,74b99a51-bb65-436f-8bcc-7022af2a490a
2,Alonzo Mourning,8051eb29-ecd7-46b5-b2fa-415777e11e83
3,Juwan Howard,5bdd7fb5-050d-4b0c-befb-5fbf7109397c
4,Hakeem Olajuwon,be9d9c53-d445-4b8c-acc9-ad1c30e8598b
...,...,...
2224,Jordan Schakel,49852ad9-c5e3-4124-bd75-2ad27b5a859f
2225,Jordan Goodwin,70d9ad35-fb19-497a-a358-d77a2a2e89ba
2226,Craig Sword,c29acdbe-9077-44c0-a8b3-f997ddff72a2
2227,Jaime Echenique,727da240-fbce-4643-ba94-f74509e2e746


In [4]:
null_players = pd.read_csv('../updated_datasets/null_players.csv')
null_players = null_players[null_players.columns[1:]]
null_players.head()

,New_name,Original_name
0,Darrell Armstrong,Darrel Armstrong
1,Predrag Stojakovic,Peja Stojakovic
2,PJ Brown,P.J. Brown
3,Jaren Jackson Jr,Jaren Jackson
4,AJ Guyton,A.J. Guyton


In [3]:
year = '2022'
draft_df = pd.read_html(requests.get(f'https://www.basketball-reference.com/draft/NBA_{year}.html').text)[0]
draft_df.columns = draft_df.columns.get_level_values(1)
draft_df = draft_df[draft_df.columns[1:4]]
draft_df.dropna(inplace=True)
draft_df = draft_df[draft_df['Player'] != 'Player']
draft_df.rename(columns={'Pk':'Pick','Tm':'Team'}, inplace=True)
draft_df['Year'] = year
draft_df.head()

,Pick,Team,Player,Year
0,1,ORL,Paolo Banchero,2022
1,2,OKC,Chet Holmgren,2022
2,3,HOU,Jabari Smith Jr.,2022
3,4,SAC,Keegan Murray,2022
4,5,DET,Jaden Ivey,2022


In [5]:
table_list = []
df_list = []
null_list = []
def draft_data(num):
    for i in range(num):
        year = 2000 + i
        var_name = f'draft_{year}'
        table_list.append(var_name)
        
        #grab the datset
        globals()[var_name] = pd.read_html(requests.get(f'https://www.basketball-reference.com/draft/NBA_{year}.html').text)[0]
        
        #clean the data
        globals()[var_name].columns = globals()[var_name].columns.get_level_values(1)
        globals()[var_name] = globals()[var_name][globals()[var_name].columns[1:4]]
        globals()[var_name].dropna(inplace=True)
        globals()[var_name] = globals()[var_name][globals()[var_name]['Player'] != 'Player']
        globals()[var_name].rename(columns={'Pk':'Pick','Tm':'Team'}, inplace=True)
        globals()[var_name]['Year'] = year

        #gather uuids
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        
        #fix mispelled names 
        for bad_name in globals()[var_name]['Player']:
            if bad_name in [x for x in null_players['Original_name']]:
                globals()[var_name].loc[globals()[var_name][globals()[var_name]['Player'] == bad_name].index, 'Player'] = [
                    x for x in null_players[null_players['Original_name'] == bad_name]['New_name']][0]
        
        #merge w/ index table to add unique ids and save csvs
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        globals()[var_name].drop(columns=['UUID_x'], inplace=True)
        globals()[var_name].rename(columns={'UUID_y':'UUID'}, inplace=True)
                
        #clean remaining nulls
        globals()[var_name].dropna(inplace=True)
        
        globals()[var_name].to_csv(f'../updated_datasets/draft_data/{var_name}.csv')
        
        #add dataframe to my output list of dataframes
        df_list.append(globals()[var_name])

    return table_list

In [6]:
draft_data(22)

['draft_2000',
 'draft_2001',
 'draft_2002',
 'draft_2003',
 'draft_2004',
 'draft_2005',
 'draft_2006',
 'draft_2007',
 'draft_2008',
 'draft_2009',
 'draft_2010',
 'draft_2011',
 'draft_2012',
 'draft_2013',
 'draft_2014',
 'draft_2015',
 'draft_2016',
 'draft_2017',
 'draft_2018',
 'draft_2019',
 'draft_2020',
 'draft_2021']

In [14]:
draft_2000.head()

,Pick,Team,Player,Year,UUID
0,1,NJN,Kenyon Martin,2000,d942225a-2af7-4ddd-ab29-5080cf48f46c
1,2,VAN,Stromile Swift,2000,b3076aff-68f5-4de2-9da0-f0cf36db15b7
2,3,LAC,Darius Miles,2000,c932ca62-defd-4f5e-8be1-9e05b04ce727
3,4,CHI,Marcus Fizer,2000,000067f7-aad4-483c-ad54-00c846384e72
4,5,ORL,Mike Miller,2000,263bcbf1-6373-464f-a26f-b58b8b2a96aa


In [7]:
for i in range(22):
    var = f'draft_{2000 + i}'
    print(f'{var} has {globals()[var].isnull().sum()[0]} nulls')

draft_2000 has 0 nulls
draft_2001 has 0 nulls
draft_2002 has 0 nulls
draft_2003 has 0 nulls
draft_2004 has 0 nulls
draft_2005 has 0 nulls
draft_2006 has 0 nulls
draft_2007 has 0 nulls
draft_2008 has 0 nulls
draft_2009 has 0 nulls
draft_2010 has 0 nulls
draft_2011 has 0 nulls
draft_2012 has 0 nulls
draft_2013 has 0 nulls
draft_2014 has 0 nulls
draft_2015 has 0 nulls
draft_2016 has 0 nulls
draft_2017 has 0 nulls
draft_2018 has 0 nulls
draft_2019 has 0 nulls
draft_2020 has 0 nulls
draft_2021 has 0 nulls
